Acá importamos los csv, hice particioes de el csv titulados para que se maneje en entorno github sin problemas de tamaño

In [ ]:
import pandas as pd
import numpy as np
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


# Leer el archivo CSV de titulados ed_sup 2022
part1 = pd.read_csv('part1.csv')
part2 = pd.read_csv('part2.csv')
part3 = pd.read_csv('part3.csv')
part4 = pd.read_csv('part4.csv')
df_titulados = pd.concat([part1, part2, part3, part4])

In [ ]:
df_titulados.head(3)

Ahora importamos los csv de notas y los concatenamos entre ellos

In [ ]:
df_notas2015 = pd.read_csv('20240404 Notas y Egresados EnseñanzaMedia 2015_WEB.csv',sep=";", encoding='latin1')
df_notas2016 = pd.read_csv('20240404 Notas y Egresados EnseñanzaMedia 2016_WEB.csv',sep=";", encoding='latin1')
df_notas2017 = pd.read_csv('20240404 Notas y Egresados EnseñanzaMedia 2017_WEB.csv',sep=";", encoding='latin1')
df_notas2018 = pd.read_csv('20240404 Notas y Egresados EnseñanzaMedia 2018_WEB.csv',sep=";", encoding='latin1')
df_notas2019 = pd.read_csv('20240404 Notas y Egresados EnseñanzaMedia 2019_WEB.csv',sep=";", encoding='latin1')
df_notas2020 = pd.read_csv('20240404 Notas y Egresados EnseñanzaMedia 2020_WEB.csv',sep=";", encoding='latin1')
df_notas2015_2020 = pd.concat([df_notas2015, df_notas2016, df_notas2017, df_notas2018, df_notas2019, df_notas2020])


In [ ]:
df_notas2015_2020.head()

Preprocesado titulados

In [ ]:
#vemos el tamaño para saber las dimensiones en futuras referencias
df_titulados.shape

In [ ]:
#evaluamos las columnas para ver que utilizaremos
df_titulados.head(5)

In [ ]:
#decidimos eliminar las columnas que no vemos util
columnas_a_eliminar = ['fec_nac_alu','anio_ing_carr_ori','sem_ing_carr_ori','sem_ing_carr_act','fecha_obtencion_titulo','cod_inst','cod_sede','cod_carrera','dur_proceso_tit','provincia_sede','comuna_sede','version','tipo_plan_carr','area_cineunesco','area_cine_f_97','subarea_cine_f_97','area_cine_f_13','subarea_cine_f_13','area_carrera_generica_n']
df_titulados.drop(columns=columnas_a_eliminar, inplace=True)

In [ ]:
#evaluamos si hay nulos
df_titulados.isnull().sum()

In [ ]:
#Notamos que mrun tiene nulos lo cual no lo consideraremos ya que son necesarios para un merge a futuro que haremos
df_titulados.dropna(subset=["mrun"], inplace=True)

In [ ]:
#tambien notamos que hay muchos nulos en nombres de itulos obtenidos, lo que asumimos que no lo consiguieron.
# por lo que lo rellenaremos como no completados
df_titulados["nomb_titulo_obtenido"].fillna('No completado', inplace=True)

In [ ]:
#lo mismo para el nombre de grado obtenido
df_titulados["nomb_grado_obtenido"].fillna('Grado superior no obtenido', inplace=True)

In [ ]:
#comprobamos si hay duplicados
df_titulados.duplicated().sum()
df_titulados.drop_duplicates

In [ ]:
#ahora vemos si los tippos de datos son los correctos en las columnas y si son necesarios cambiarlos 
df_titulados.info()

In [ ]:
#necesitamos el mrun como int para el merge

# Asegurarse de que sea de tipo int64
df_titulados['mrun'] = df_titulados['mrun'].astype(np.int64)

# Verificar el tipo de datos de la columna
print("verificamos si es de tipo int64:", df_titulados['mrun'].dtype)


Preprocesado notas

In [ ]:
df_notas2015_2020.shape

In [ ]:
df_notas2015_2020.head(20)

In [ ]:
#decidimos eliminar las columnas que no vemos util
columnas_a_eliminar = ['DGV_RBD','COD_REG_RBD','RBD','COD_PRO_RBD','COD_COM_RBD','NOM_COM_RBD','COD_DEPROV_RBD','NOM_DEPROV_RBD']
df_notas2015_2020.drop(columns=columnas_a_eliminar, inplace=True)

In [ ]:
#evaluamos si hay nulos
df_notas2015_2020.isnull().sum()

In [ ]:
#comprobamos si hay duplicados
df_notas2015_2020.duplicated().sum()

In [ ]:
#ahora vemos si los tippos de datos son los correctos en las columnas y si son necesarios cambiarlos 
# (veremos en el codigo mas adelante si son necesarios)
df_notas2015_2020.info()

Pensando en el merge

hacemos merge entre los 2 df concatenados y preprocesados

In [ ]:
df_merge_titulados_notas=pd.merge(df_notas2015_2020,df_titulados, left_on="MRUN",right_on="mrun")

In [ ]:
df_merge_titulados_notas

In [ ]:
#decidimos eliminar las columnas que no vemos util, hacemos una version con regiones que usaremos mas adelante
columnas_a_eliminar = ['COD_GRADO','MRUN','codigo_unico','mrun','rango_edad','nomb_titulo_obtenido','tipo_inst_1','tipo_inst_2','tipo_inst_3','nomb_inst','nomb_sede','nomb_carrera','nivel_global','dur_estudio_carr','dur_total_carr','region_sede','jornada','modalidad','ORIGEN','nivel_carrera_1','nomb_grado_obtenido']
df_regiones=df_merge_titulados_notas.drop(columns=columnas_a_eliminar)


#decidimos eliminar las columnas que no vemos util
columnas_a_eliminar = ['COD_GRADO','MRUN','codigo_unico','mrun','rango_edad','nomb_titulo_obtenido','tipo_inst_1','tipo_inst_2','tipo_inst_3','nomb_inst','nomb_sede','nomb_carrera','nivel_global','dur_estudio_carr','dur_total_carr','region_sede','jornada','modalidad','ORIGEN','nivel_carrera_1','nomb_grado_obtenido','NOM_REG_RBD_A']
df_merge_titulados_notas.drop(columns=columnas_a_eliminar, inplace=True)


In [ ]:
df_merge_titulados_notas=pd.merge(df_notas2015_2020,df_titulados, left_on="MRUN",right_on="mrun")

In [ ]:

#decidimos eliminar las columnas que no vemos util
columnas_a_eliminar = ['COD_GRADO','MRUN','codigo_unico','mrun','rango_edad','nomb_titulo_obtenido','tipo_inst_1','tipo_inst_2','tipo_inst_3','nomb_inst','nomb_sede','nomb_carrera','nivel_global','dur_estudio_carr','dur_total_carr','region_sede','jornada','modalidad','ORIGEN','nivel_carrera_1','nomb_grado_obtenido']
df_regiones=df_merge_titulados_notas.drop(columns=columnas_a_eliminar)


In [ ]:
df_merge_titulados_notas=pd.merge(df_notas2015_2020,df_titulados, left_on="MRUN",right_on="mrun")

In [ ]:
# Eliminar las filas donde ENSE_COMPLETA es 0 asegurandonos de tratar con casos de egresados
df_merge_titulados_notas.drop(df_merge_titulados_notas[df_merge_titulados_notas['MARCA_EGRESO'] == 0].index, inplace=True)

# Verificar el resultado
df_merge_titulados_notas[df_merge_titulados_notas["MARCA_EGRESO"]==0]


In [ ]:
# Eliminar las filas donde ENSE_COMPLETA es 0 asegurandonos de tratar con casos de egresados
df_regiones.drop(df_regiones[df_regiones['MARCA_EGRESO'] == 0].index, inplace=True)

# Verificar el resultado
df_regiones[df_regiones["MARCA_EGRESO"]==0]

In [ ]:
df_merge_titulados_notas.isnull().sum()

In [ ]:
#tener ojo con los tipos de datos para futuras referencias de tipos de datos
df_merge_titulados_notas.info()

In [ ]:
# Reemplazar comas por puntos
df_merge_titulados_notas['PROM_NOTAS_ALU'] = df_merge_titulados_notas['PROM_NOTAS_ALU'].str.replace(',', '.')

# Asegurarse de que la columna sea de tipo float64
df_merge_titulados_notas['PROM_NOTAS_ALU'] = df_merge_titulados_notas['PROM_NOTAS_ALU'].astype(np.float64)

# Verificar el tipo de datos de la columna
print("verificamos si es de tipo float64:", df_merge_titulados_notas['PROM_NOTAS_ALU'].dtype)

Aca exportamos el dataframe preprocesado para trabajar en entorno colab

In [ ]:

df_merge_titulados_notas.to_csv('df_merge_titulados_notas.csv', index=False)

In [ ]:
df_regiones.to_csv('df_regiones.csv', index=False)